# Embed a holoviews interactive chart in a folium map (0.0.1-alpha)

The out out of this notebook can be seen at

https://sfsheath.github.io/holoviews-in-folium/embedded_iframe.html

All very "quick and dirty". Is the an approach that doesn't require writing holoviews variable to a file? I tried using a hv.Renderer object, but the 'raw' html that generates has dependencies that are met when it's written out to an html file.

In [ ]:
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
hv.extension('bokeh')
# hv.extension('mapplotlib')

from bokeh.models import HoverTool
from folium import FeatureGroup, LayerControl, Map, Marker



from IPython.display import HTML,IFrame

%matplotlib inline

## Load data and simple pandas plotting

In [ ]:
ramphs_df = pd.read_csv('roman-amphitheaters.csv')
chronogrps_df = pd.read_csv('chronogrps.csv')

In [ ]:
# create a holoviews chart, any holoviews chart. 

cap_summed_df = ramphs_df.groupby(by = 'modcountry', as_index=False).sum()\
   .sort_values(by = 'capacity', ascending = False)

cap_hover = HoverTool(tooltips=[("capacity", "@capacity{int}")])
    
imp_bars = hv.Bars(cap_summed_df[['modcountry', 'capacity']])\
.options(height=300, fill_color='pink', color='pink', xrotation = 90)




In [ ]:
%%output filename="embed_bokeh" fig="html" 
# I'd like to skip this step. Can I?? Will keep working on it.

hv.extension('bokeh')
imp_bars.options(xrotation = 90,tools=[cap_hover])

In [ ]:
# make a folium map. 
m = folium.Map([46, 14], zoom_start=4)

# get the holoviews out as a variable
with open('embed_bokeh.html', 'r') as myfile:
  data = myfile.read()

html = data

# create and iframe than a popup with that iframe as content
iframe = folium.IFrame(html=html, width=300, height=300)
popup = folium.Popup(iframe, max_width=2650)


folium.Marker([46, 14], popup=popup).add_to(m)



m.save("embedded_iframe.html", )
m
